<a href="https://colab.research.google.com/github/ShimaSama/Netflix-QoS/blob/main/google_colab/SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import metrics
from sklearn.svm import SVR
import random
import joblib

# Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = '/content/drive/My Drive/MALWIAS/TMA/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_dataset_from_csv(filename):
  file = open(BASE_DIR + filename, 'r')
  custom_fields = {item: lambda x: int(x, 16) for item in ["tcp.flags", "tcp.checksum", "ip.checksum", "ip.dsfield", "ip.flags", "ip.id"]}
  custom_fields.update({'label': lambda x: float(x == "netflix")})
  dataset = pandas.read_csv(file, converters=custom_fields)
  values = dataset.values
  X = values[:,:len(values[1]) - 1]
  Y = values[:,len(values[1]) - 1]
  return X, Y

X, Y = get_dataset_from_csv('data/computer.csv')
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.3, random_state=random.randint(0, 100000))
X_test, Y_test = get_dataset_from_csv('data/test.csv')

# Create Model

## Loading model

In [ ]:
filename = BASE_DIR + 'models/SVR_model.sav'
model = joblib.load(filename)

FileNotFoundError: ignored

## Creating model

In [ ]:
model = SVR(verbose=True)
model.fit(X_train, Y_train)
plt.bar(range(len(model.feature_importances_)), model.feature_importances_)
plt.show()

[LibSVM]

## Saving model

In [ ]:
filename = BASE_DIR + 'models/SVR_model.sav'
joblib.dump(model, filename)

FileNotFoundError: ignored

# Test & Validation

In [ ]:
score = model.score(X_validation, Y_validation)
expected = Y_test
predicted = model.predict(X_test)

print("Model Score: %s" % score)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

NameError: ignored

In [ ]:
th = 0.5
false_positive = false_negative = true_positive = true_negative = good = 0
for predicted_i, expected_i in zip(predicted, expected):
  predicted_val = int(predicted_i >= th)
  if predicted_val == int(expected_i):
    good += 1
    if predicted_val == 0:
      true_negative += 1
    else:
      true_positive += 1
  else:
    if predicted_val == 0:
      false_negative += 1
    else:
      false_positive += 1

print("true positive = " + str(true_positive)+ "---> rate = " + str(true_positive/(true_positive+false_negative)))
print("true negative = " + str(true_negative) + "---> rate = " + str(true_negative/(true_negative+false_positive)))
print("false negative = " + str(false_negative) + "---> rate = " + str(false_negative/(false_negative+true_positive)))
print("false positive = " + str(false_positive) + "---> rate = " + str(false_positive/(false_positive+true_negative)))
print("sensitivity = " + str(true_positive/(true_positive+false_negative)))
print("specifity = " + str(true_negative/(true_negative+false_positive)))
print("acurracy = " + str(good/len(predicted)))

true positive = 749---> rate = 0.27833519137866963
true negative = 17794---> rate = 0.8843936381709742
false negative = 1942---> rate = 0.7216648086213303
false positive = 2326---> rate = 0.11560636182902584
sensitivity = 0.27833519137866963
specifity = 0.8843936381709742
acurracy = 0.8128972863969137
